In [6]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
training_data_gen = ImageDataGenerator(rotation_range=0.2,shear_range=0.2,
zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2,rescale=1./255,
validation_split = 0.2) #created 5 new data points or images from an existing image.
#rescaling because the scale of the original image is way too large.

In [13]:
training_data = training_data_gen.flow_from_directory(r'PreparingData\Train',
target_size=(80,80),batch_size=8,class_mode='categorical',subset = 'training')

validation_data = training_data_gen.flow_from_directory(r'PreparingData\Train',
target_size=(80,80),batch_size=8,class_mode='categorical',subset = 'validation')
#we need to give a particular size to the image then only our model will work.

Found 64244 images belonging to 2 classes.
Found 16060 images belonging to 2 classes.


In [15]:
test_data_gen = ImageDataGenerator(rotation_range=0.2)

test_data = test_data_gen.flow_from_directory(r'PreparingData\Test',
target_size=(80,80),batch_size=8,class_mode='categorical')

Found 4594 images belonging to 2 classes.


In [18]:
#Making the base of the model
model_base = InceptionV3(include_top= False,weights='imagenet',
input_tensor=Input(shape=(80,80,3),batch_size = 8))
#3 due to lightning conditions

87924736/87910968 [==============================] - 78s 1us/step


In [19]:
my_model = model_base.output

In [21]:
model_base.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(8, 80, 80, 3)]     0           []                               
                                                                                                  
 conv2d (Conv2D)                (8, 39, 39, 32)      864         ['input_2[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (8, 39, 39, 32)     96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (8, 39, 39, 32)      0           ['batch_normalization[

In [24]:
my_model = Flatten()(my_model)
#deep learning starts from here
my_model = Dense(64,activation='relu')(my_model)
my_model = Dropout(0.5)(my_model)
my_model = Dense(2,activation='softmax')(my_model)
#since this is a categorical model so activation will be softmax.
 #just put those 2 neurons(eye_state)
#drop out to prevent overfitting
#activation= relu means take only important data you can say.

In [26]:
#final model
model = Model(inputs=model_base.input, outputs =my_model)
#since we donot want to train the model from scratch so
for layer in model_base.layers:
    layer.trainable = False

In [27]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(8, 80, 80, 3)]     0           []                               
                                                                                                  
 conv2d (Conv2D)                (8, 39, 39, 32)      864         ['input_2[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (8, 39, 39, 32)     96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (8, 39, 39, 32)      0           ['batch_normalization[0][0]']